In [ ]:
print('OK')

In [1]:
%pwd

'f:\\Alhussien Ayman\\courses\\Programming\\Projects\\Medical_Chatbot\\research'

In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'f:\\Alhussien Ayman\\courses\\Programming\\Projects\\Medical_Chatbot'

In [3]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Extract data from the PDF file 

def load_pdf_file (data) :
    loader =DirectoryLoader(data, glob='*.pdf',loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [5]:
extracted_data = load_pdf_file(data='data/')

In [ ]:
extracted_data

In [6]:
# split data into text chunks
def text_split(extracted_data) :
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500 , chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
text_chunks = text_split(extracted_data)
print('Length of text chunks', len(text_chunks))

Length of text chunks 6089


In [9]:
def download_hugging_face_embedding () :
    embeddings = HuggingFaceEmbeddings (model_name = 'sentence-transformers/all-MiniLM-L6-v2', model_kwargs={"device": "cuda"})
    
    return embeddings

In [10]:
embeddings= download_hugging_face_embedding()

C:\Users\ALMASA\AppData\Local\Temp\ipykernel_10580\1428725927.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings (model_name = 'sentence-transformers/all-MiniLM-L6-v2', model_kwargs={"device": "cuda"})
C:\Users\ALMASA\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\ALMASA\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ALMASA\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fal

In [11]:
query_result = embeddings.embed_query("Hello world")
print('Length', len(query_result))

Length 384


In [65]:
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY =os.environ.get('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN =os.environ.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
from pinecone import Pinecone, ServerlessSpec
import os
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension= 384 ,
        metric='cosine' ,
        spec = ServerlessSpec (
            cloud='aws',
            region='us-east-1'
        )
    )

In [67]:
import os 
os.environ['PINCONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [68]:
from langchain_pinecone import PineconeVectorStore

docdearch = PineconeVectorStore.from_documents(
    documents=text_chunks ,
    index_name=index_name,
    embedding=embeddings
)

In [69]:
from langchain_pinecone import PineconeVectorStore
docdearch = PineconeVectorStore.from_existing_index (
    index_name=index_name ,
    embedding=embeddings
)

In [70]:
docdearch

In [71]:
retriever = docdearch.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [72]:
retriever_docs =retriever.invoke('What is Acne ?')

In [73]:
retriever_docs

[Document(id='8d97dc08-6539-49b2-8dc2-baed1cc0484b', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-08-27T12:22:44+00:00', 'page': 55.0, 'page_label': '56', 'producer': 'iLovePDF', 'source': 'data\\Medical_book.pdf', 'total_pages': 700.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='0145b64f-ae7d-4685-ad34-5e3c100dfc1d', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-08-27T12:22:44+00:00', 'page': 55.0, 'page_label': '56', 'producer': 'iLovePDF', 'source': 'data\\Medical_book.pdf', 'total_pages': 700.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='1d1f8b29-8ff2-4aa6-9d44-0c276235ead6', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-08-27T12:22:44+00:00', 'page': 55.0, 'page_label': '56', 'producer': 'iLovePDF', 'source': 'data\\Medical_book.pdf', 'total_pages': 700.0}, page_content='Se

In [74]:
from langchain_openai import OpenAI 
llm =OpenAI(temperature=0.4 , max_tokens=500)

In [75]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    " You are an assistant for question-answering tasks."
    'Use the following pieces of retrieved context to answer'
    'the question . If you do not know the answer , say that you '
    'do not know . Use three sentences maximum and keep the '
    'answer concise.'
    '\n\n'
    '{context}'
)

prompt = ChatPromptTemplate.from_messages (
    [
        ('system',system_prompt),
        ('human','{input}')
    ]
)

In [76]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [77]:
response = rag_chain.invoke({'input' : 'what is Acne?'})
print(response['answer'])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}